In [1]:
from dotenv import load_dotenv
load_dotenv(dotenv_path='.env')

True

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain

chat_model = ChatOpenAI()

conversation_chain = ConversationChain(
    llm=chat_model,
)

print(conversation_chain.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


# Few-shot learning

In [3]:
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate

examples = [
    {"input": "2+2", "output": "4"},
    {"input": "3+2", "output": "5"},
    {"input": "5+6", "output": "11"},
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

print(few_shot_prompt.format())

Human: 2+2
AI: 4
Human: 3+2
AI: 5
Human: 5+6
AI: 11


In [4]:
final_prompt = few_shot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a wonderous wizard of math."),
        ("human", "{input}"),
    ]
)

print(final_prompt.format(input="2+2"))

System: You are a wonderous wizard of math.
Human: 2+2


In [5]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=chat_model,
    prompt=final_prompt,
)

chain.run("What is 3+3?")

'The sum of 3 and 3 is 6.'

# Memetic Proxy

In [6]:
from langchain.prompts import PromptTemplate

template = """
System: {reference}
Provide a helpful answer to the following question

Human: {question}

AI:
"""

chain = LLMChain(
    llm=chat_model,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

high_level = "Imagine you are a Professor teching at PhD level."
lower_level = "Imagine you are kindergarten teacher."

question = "What is a quantum mechanics?"

chain.run({
    "reference": high_level,
    "question": question,
})




> Entering new LLMChain chain...
Prompt after formatting:

System: Imagine you are a Professor teching at PhD level.
Provide a helpful answer to the following question

Human: What is a quantum mechanics?

AI:


> Finished chain.


'Quantum mechanics is a fundamental theory in physics that describes the behavior of particles and systems at the smallest scales, such as atoms and subatomic particles. It provides a mathematical framework to understand and predict the behavior of these particles, which often exhibit wave-particle duality and can be in superposition states. Quantum mechanics is crucial for understanding various phenomena, such as the behavior of electrons in atoms, the interaction of light with matter, and the principles underlying quantum computing and cryptography. It is a highly complex and abstract field, but its applications have revolutionized numerous areas of science and technology.'

In [7]:
chain.run({
    "reference": lower_level,
    "question": question,
})



> Entering new LLMChain chain...
Prompt after formatting:

System: Imagine you are kindergarten teacher.
Provide a helpful answer to the following question

Human: What is a quantum mechanics?

AI:


> Finished chain.


'A quantum mechanics is a branch of physics that studies the behavior of particles at the atomic and subatomic level. It deals with concepts such as wave-particle duality, superposition, and entanglement. In simpler terms, it explores the strange and fascinating world of extremely small particles and their interactions. While it may be a complex topic, it is not something that would typically be discussed in a kindergarten classroom.'

# Chain of thought

Dataset: https://www.kaggle.com/datasets/konradb/chain-of-thought-collection

In [8]:
import pandas as pd

cot_df = pd.read_csv("./other_data/CoT_collection.csv")

cot_df

,Question,Rationale,Response
0,Blake filled a bucket with 0.8 gallon of water...,Since Blake filled the bucket with 0.8 gallon ...,0.6
1,There is 0.2 cup of oil in Scarlett's measurin...,0.2 + 0.3 = 0.5,0.5
2,Craig walked 0.2 mile from school to David's h...,Craig walked 0.2 mile from school to David's h...,0.9
3,Hannah's Vegetarian Restaurant bought 0.3 poun...,Hannah's Vegetarian Restaurant bought 0.3 poun...,0.9
4,Alyssa bought some toys. She bought a football...,Alyssa spent $5.71 on football and $6.59 on ma...,12.3
...,...,...,...
575,"Ella owns 2 dogs. Each day, 1 dog eats 0.175 s...","Ella owns 2 dogs. Each day, 1 dog eats 0.175 s...",0.3
576,"Mike found 6 seashells and 4 starfish, but 4 o...",The problem is: 6 + 4 - 4 / 2.0 = 10.0 - 2.0 =...,2.0
577,Mary loves eating fruits. Mary paid $11.08 for...,Mary loves eating fruits. Mary paid $11.08 for...,34.7
578,Joan has 40 blue balloons Melanie has 41 blue ...,Joan has 40 blue balloons Melanie has 41 blue ...,81.0


In [9]:
list(cot_df.T.to_dict().values())[0]

{'Question': 'Blake filled a bucket with 0.8 gallon of water. Later, he poured out 0.2 gallon of the water. How much water is in the bucket?',
 'Rationale': 'Since Blake filled the bucket with 0.8 gallon of water, and poured out 0.2 gallon of it, the current amount of water left in the bucket is 0.8 - 0.2 = 0.6 gallon',
 'Response': 0.6}

In [10]:
import random

random.sample(list(cot_df.T.to_dict().values()), 10)

[{'Question': 'At the hardware store, 0.2 the nails are size 2d and 0.5 the nails are size 4d. What fraction of the nails are either size 2d or 4d?',
  'Rationale': 'At the hardware store, 0.2 the nails are size 2d and 0.5 the nails are size 4d. Thus, (0.2 + 0.5) of all nails = (0.7) of all nails',
  'Response': 0.7},
 {'Question': 'Sally had 760 quarters in her bank. She spent 418 of her quarters.How many quarters does she have now?',
  'Rationale': 'To solve the problem, we can use the following steps:\\n1. Subtract the number of quarters spent from total number of quarters to find out how many quarters Sally has left:\\n760 - 418 = 342.\\n2. Round up to one decimal place: 342.0',
  'Response': 342.0},
 {'Question': 'Greg and Sharon own neighboring cornfields. Greg harvested 0.4 acreof corn on Monday and Sharon harvested 0.1 acre. How many moreacres did Greg harvest than Sharon?',
  'Rationale': 'Greg harvested 0.4 acre of corn on Monday and Sharon harvested 0.1acre. The difference i

In [11]:
from langchain.prompts import (
    PromptTemplate,
    FewShotPromptTemplate,
)

random_samples = random.sample(list(cot_df.T.to_dict().values()), 10)

template = """
Question: {Question}
Rationale: {Rationale}
Reponse: {Response}
"""

example_prompt = PromptTemplate(
    template=template,
    input_variables=["Question", "Rationale", "Response"],
)

system_prompt = """
You are provided with an arithmetic question.
Your task is to compute the solution using the given arithmetic operations.
The only arithmetic operations needed to answer the question are +(addition), -(subtraction).
The answer should be correct to one decimal place.
"""

COT_Prompt = FewShotPromptTemplate(
    prefix=system_prompt,
    example_prompt=example_prompt,
    examples=random_samples,
    suffix="Question: {input}",
    input_variables=["input"],
)

COT_Prompt.format(input="EXAMPLE Question")

'\nYou are provided with an arithmetic question.\nYour task is to compute the solution using the given arithmetic operations.\nThe only arithmetic operations needed to answer the question are +(addition), -(subtraction).\nThe answer should be correct to one decimal place.\n\n\n\nQuestion: Roadster\'s Paving Company used 10 tons of cement to pave Lexi\'sstreet and 5.1 tons of cement to pave Tess\'s street. How muchcement did Roadster\'s Paving Company use in all?\nRationale: Roadster\'s Paving Company used 10 tons of cement to pave Lexi\'sstreet and 5.1 tons of cement to pave Tess\'s street. How muchcement did Roadster\'s Paving Company use in all?\n\n10 + 5.1 = 15.1\nReponse: 15.1\n\n\n\nQuestion: Ella owns 2 dogs. Each day, 1 dog eats 0.175 scoop of dog food andthe other dog eats 0.125 scoop. Together, how much dog food dothe 2 dogs eat each day?\nRationale: Ella owns 2 dogs. Each day, 1 dog eats 0.175 scoop of dog food and the other dog eats 0.125 scoop. So, together, the 2 dogs eat 

In [12]:
from langchain.chains import LLMChain

chain = LLMChain(
    llm=chat_model,
    prompt=COT_Prompt,
    verbose=True,
)


chain.run("""
Mandy made an apple pie. She used 0.6 teaspoons of cinnamon and 0.4 teaspoons of nutmeg.
          
How much more cinnamon than nutmeg did Mandy use?""")



> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question.
Your task is to compute the solution using the given arithmetic operations.
The only arithmetic operations needed to answer the question are +(addition), -(subtraction).
The answer should be correct to one decimal place.



Question: Roadster's Paving Company used 10 tons of cement to pave Lexi'sstreet and 5.1 tons of cement to pave Tess's street. How muchcement did Roadster's Paving Company use in all?
Rationale: Roadster's Paving Company used 10 tons of cement to pave Lexi'sstreet and 5.1 tons of cement to pave Tess's street. How muchcement did Roadster's Paving Company use in all?

10 + 5.1 = 15.1
Reponse: 15.1



Question: Ella owns 2 dogs. Each day, 1 dog eats 0.175 scoop of dog food andthe other dog eats 0.125 scoop. Together, how much dog food dothe 2 dogs eat each day?
Rationale: Ella owns 2 dogs. Each day, 1 dog eats 0.175 scoop of dog food and the other dog eats 0.125 sc

'Rationale: Mandy used 0.6 teaspoons of cinnamon and 0.4 teaspoons of nutmeg. To find out how much more cinnamon than nutmeg Mandy used, we subtract the amount of nutmeg from the amount of cinnamon: 0.6 - 0.4 = 0.2\nReponse: 0.2'

# Self-consistency

In [13]:
chat_model = ChatOpenAI(temperature=0.7)

cot_chain = LLMChain(
    llm=chat_model,
    prompt=COT_Prompt,
    verbose=True,
)

template = """
Using the following answers, return the answer that comes up the most often.

{answers}

ANSWER:
"""

consistency_prompt = PromptTemplate.from_template(template)

consistency_chain = LLMChain(
    llm=chat_model,
    prompt=consistency_prompt,
    verbose=True,
)


question = """
Mandy made an apple pie. She used 0.6 teaspoons of cinnamon and 0.5 teaspoons of nutmeg.
How much more cinnamon than nutmeg did Mandy use?
"""

responses = []

for i in range(10):
    responses.append(cot_chain.run(question))

answers = "\n".join(responses)

consistency_chain.run(answers)



> Entering new LLMChain chain...
Prompt after formatting:

You are provided with an arithmetic question.
Your task is to compute the solution using the given arithmetic operations.
The only arithmetic operations needed to answer the question are +(addition), -(subtraction).
The answer should be correct to one decimal place.



Question: Roadster's Paving Company used 10 tons of cement to pave Lexi'sstreet and 5.1 tons of cement to pave Tess's street. How muchcement did Roadster's Paving Company use in all?
Rationale: Roadster's Paving Company used 10 tons of cement to pave Lexi'sstreet and 5.1 tons of cement to pave Tess's street. How muchcement did Roadster's Paving Company use in all?

10 + 5.1 = 15.1
Reponse: 15.1



Question: Ella owns 2 dogs. Each day, 1 dog eats 0.175 scoop of dog food andthe other dog eats 0.125 scoop. Together, how much dog food dothe 2 dogs eat each day?
Rationale: Ella owns 2 dogs. Each day, 1 dog eats 0.175 scoop of dog food and the other dog eats 0.125 sc

'0.1'

# Inception

In [14]:
system_prompt = """
You are provided with an arithmetic question.
Your task is to compute the solution using the given arithmetic operations.
The only arithmetic operations needed to answer the question are +(addition), -(subtraction).
The answer should be correct to one decimal place.
"""

template = """
{input}

Let's think about it step by step.
"""

chain = LLMChain(
    llm=chat_model,
    prompt=PromptTemplate.from_template(template),
    verbose=True,
)

question = """
Mandy made an apple pie. She used 0.6 teaspoons of cinnamon and 0.5 teaspoons of nutmeg.
How much more cinnamon than nutmeg did Mandy use?
"""

chain.run(question)




> Entering new LLMChain chain...
Prompt after formatting:


Mandy made an apple pie. She used 0.6 teaspoons of cinnamon and 0.5 teaspoons of nutmeg.
How much more cinnamon than nutmeg did Mandy use?


Let's think about it step by step.


> Finished chain.


"First, let's find the difference between the amounts of cinnamon and nutmeg used.\nTo do this, we subtract the amount of nutmeg used from the amount of cinnamon used: 0.6 teaspoons - 0.5 teaspoons = 0.1 teaspoons.\n\nTherefore, Mandy used 0.1 teaspoons more cinnamon than nutmeg."

# Self-ask

In [15]:
from langchain.agents import (
    initialize_agent,
    AgentType,
    load_tools,
)

tools = load_tools(["serpapi"])
tools[0].name = "Intermediate Answer"

self_ask_with_search = initialize_agent(
    tools=tools,
    llm=chat_model,
    agent=AgentType.SELF_ASK_WITH_SEARCH,
    verbose=True,
)

print(self_ask_with_search.agent.llm_chain.prompt.template)

Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mother of George Washington was Mary Ball Washington.
Follow up: Who was the father of Mary Ball Washin

In [18]:
from langchain.callbacks import StdOutCallbackHandler
handler = StdOutCallbackHandler()

question = "What is the hometown of the president of reigning men's U.S. Open champion?"

self_ask_with_search.run(
    question,
    callbacks=[handler],
)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Question: Who lived longer, Muhammad Ali or Alan Turing?
Are follow up questions needed here: Yes.
Follow up: How old was Muhammad Ali when he died?
Intermediate answer: Muhammad Ali was 74 years old when he died.
Follow up: How old was Alan Turing when he died?
Intermediate answer: Alan Turing was 41 years old when he died.
So the final answer is: Muhammad Ali

Question: When was the founder of craigslist born?
Are follow up questions needed here: Yes.
Follow up: Who was the founder of craigslist?
Intermediate answer: Craigslist was founded by Craig Newmark.
Follow up: When was Craig Newmark born?
Intermediate answer: Craig Newmark was born on December 6, 1952.
So the final answer is: December 6, 1952

Question: Who was the maternal grandfather of George Washington?
Are follow up questions needed here: Yes.
Follow up: Who was the mother of George Washington?
Intermediate answer: The mot

'Belgrade, Serbia, Yugoslavia.'

# ReACT

In [19]:
tools = load_tools(["serpapi", "llm-math"], llm=chat_model)
agent = initialize_agent(
    tools=tools,
    llm=chat_model,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
)

handler = StdOutCallbackHandler()

question = """
Who is current Leo DeCaprio's girlfriend?
What is her age raised to the 0.43 power?
"""

agent.run(
    question,
    callbacks=[handler],
)



> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompt after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query.
Calculator: Useful for when you need to answer questions about math.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: 
Who is current Leo DeCaprio's girlfriend?
What is her age raised to the 0.43 power?

Thought:

> Finished chain.
I need to find out who Leo DeCaprio's current girlf

"The age of Leo DeCaprio's girlfriend, Vittoria Ceretti, raised to the 0.43 power is approximately 3.99."

# Plan and execute

In [20]:
from langchain_experimental.plan_and_execute import (
    PlanAndExecute,
    load_agent_executor,
    load_chat_planner
)

tools = load_tools(["serpapi", "llm-math"], llm=chat_model)

planner = load_chat_planner(llm=chat_model)

executor = load_agent_executor(tools=tools, llm=chat_model, verbose=True)

agent = PlanAndExecute(
    planner=planner,
    executor=executor,
    verbose=True,
)

In [21]:
print(planner.llm_chain.prompt.format(input="What is 2+2?"))

System: Let's first understand the problem and devise a plan to solve the problem. Please output the plan starting with the header 'Plan:' and then followed by a numbered list of steps. Please make the plan the minimum number of steps required to accurately complete the task. If the task is a question, the final step should almost always be 'Given the above steps taken, please respond to the users original question'. At the end of your plan, say '<END_OF_PLAN>'
Human: What is 2+2?


In [22]:
print(executor.chain.agent.llm_chain.prompt.format(
    previous_steps="previous_steps",
    current_step="current_step",
    agent_scratchpad="agent_scratchpad",
))

System: Respond to the human as helpfully and accurately as possible. You have access to the following tools:

Search: A search engine. Useful for when you need to answer questions about current events. Input should be a search query., args: {'tool_input': {'type': 'string'}}
Calculator: Useful for when you need to answer questions about math., args: {'tool_input': {'type': 'string'}}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or Search, Calculator

Provide only ONE action per $JSON_BLOB, as shown:

```
{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}
```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
Observation: action result
... (repeat Thought/Action/Observation N times)
Thought: I know what to respond
Action:
```
{
  "action": "Final Answer",
  "action_input": "Final response to human"
}

In [25]:
question = """
Who is current Leo DeCaprio's girlfriend?
What is her current age raised to the 0.43 power? Give numerical answer.
"""

agent.run(question)



> Entering new PlanAndExecute chain...
steps=[Step(value="Search for the latest information on Leonardo DiCaprio's girlfriend."), Step(value='Find out her current age.'), Step(value='Calculate her current age raised to the power of 0.43.'), Step(value='Round the numerical answer to an appropriate decimal place.'), Step(value="Given the above steps taken, respond to the user's original questions.\n\n")]

> Entering new AgentExecutor chain...
Action:
```
{
  "action": "Search",
  "action_input": "Leonardo DiCaprio's current girlfriend"
}
```
Observation: Vittoria Ceretti
Thought:Since the user's original question is not mentioned in the previous steps, I need to clarify what the question is before I can respond. Let me ask the user for their original question. 

Action:
```
{
  "action": "Final Answer",
  "action_input": "What was your original question?"
}
```

> Finished chain.
*****

Step: Search for the latest information on Leonardo DiCaprio's girlfriend.

Response: What was your 

'The original question was not mentioned in the previous steps. Could you please clarify what your original question was?'